<a href="https://colab.research.google.com/github/katoro8989/Kaggle/blob/signate/SIGNATE_student_cup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import seaborn as sns
from sklearn.metrics import log_loss
# from catboost import Pool, CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
import lightgbm as lgbm

import xgboost as xgb
# from tabpfn import TabPFNClassifier
from sklearn.base import BaseEstimator
from sklearn.impute import SimpleImputer
from imblearn.over_sampling import SMOTE

from sklearn.preprocessing import StandardScaler

from keras import callbacks
from keras import optimizers
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers import PReLU
from keras.layers import BatchNormalization
from keras.layers import LSTM

from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_absolute_percentage_error

# get datasets
train = pd.read_csv('/content/drive/MyDrive/SIGNATE/train.csv')
test = pd.read_csv('/content/drive/MyDrive/SIGNATE/test.csv')
sample = pd.read_csv('/content/drive/MyDrive/SIGNATE/submit_sample.csv')

# 前処理

In [ ]:
def pre_process(train, test):
  # year
  len_train = len(train)
  for id in train[train['year'] >= 2023]['id']:
    train[id:id+1]['year'] -= 1000

  for id in test[test['year'] >= 2023]['id']:
    test[id-len_train:id+1-len_train]['year'] -= 1000

  # odometer
  # trainに
  # 1000000キロ？以上のポイントが存在、マイルでもおかしい
  # -1も結構ある

  train.loc[train['odometer'] >= 1e6, 'odometer'] //= 10
  test.loc[test['odometer'] >= 1e6, 'odometer'] //= 10

  train.loc[train['odometer'] < 0, 'odometer'] = np.nan
  test.loc[test['odometer'] < 0, 'odometer'] = np.nan

  # region
  # カテゴリごとのpriceの平均値を計算
  price_mean_by_region = train.groupby('region')['price'].mean().reset_index()

  # priceの平均値が高い順にラベルを付与
  price_mean_by_region = price_mean_by_region.sort_values(by='price', ascending=True)
  price_mean_by_region['region_label'] = range(len(price_mean_by_region))

  # ラベルを元のデータにマージ
  train = train.merge(price_mean_by_region[['region', 'region_label']], on='region', how='left')
  test = test.merge(price_mean_by_region[['region', 'region_label']], on='region', how='left')

  # manufucturer

  import unicodedata

  train['manufacturer'] = train['manufacturer'].str.replace(' ', '')
  train['manufacturer'] = train['manufacturer'].apply(lambda x: unicodedata.normalize('NFKC', x))
  train['manufacturer'] = train['manufacturer'].str.lower()
  train.loc[train["manufacturer"] == "niѕsan", "manufacturer"] = "nissan"
  train.loc[train["manufacturer"] == "nisѕan", "manufacturer"] = "nissan"
  train.loc[train["manufacturer"] == "subαru", "manufacturer"] = "subaru"
  train.loc[train["manufacturer"] == "toyotа", "manufacturer"] = "toyota"
  train.loc[train["manufacturer"] == "sαturn", "manufacturer"] = "saturn"
  train.loc[train["manufacturer"] == "аcura", "manufacturer"] = "acura"
  train.loc[train["manufacturer"] == "vоlkswagen", "manufacturer"] = "volkswagen"
  train.loc[train["manufacturer"] == "lexuѕ", "manufacturer"] = "lexus"
  train.loc[train["manufacturer"] == "ᴄhrysler", "manufacturer"] = "chrysler"

  test['manufacturer'] = test['manufacturer'].str.replace(' ', '')
  test['manufacturer'] = test['manufacturer'].apply(lambda x: unicodedata.normalize('NFKC', x))
  test['manufacturer'] = test['manufacturer'].str.lower()
  test.loc[test["manufacturer"] == "niѕsan", "manufacturer"] = "nissan"
  test.loc[test["manufacturer"] == "nisѕan", "manufacturer"] = "nissan"
  test.loc[test["manufacturer"] == "subαru", "manufacturer"] = "subaru"
  test.loc[test["manufacturer"] == "toyotа", "manufacturer"] = "toyota"
  test.loc[test["manufacturer"] == "sαturn", "manufacturer"] = "saturn"
  test.loc[test["manufacturer"] == "аcura", "manufacturer"] = "acura"
  test.loc[test["manufacturer"] == "vоlkswagen", "manufacturer"] = "volkswagen"
  test.loc[test["manufacturer"] == "lexuѕ", "manufacturer"] = "lexus"
  test.loc[test["manufacturer"] == "ᴄhrysler", "manufacturer"] = "chrysler"

  # カテゴリごとのpriceの平均値を計算
  price_mean_by_region = train.groupby('manufacturer')['price'].mean().reset_index()

  # priceの平均値が高い順にラベルを付与
  price_mean_by_region = price_mean_by_region.sort_values(by='price', ascending=True)
  price_mean_by_region['manufacturer_label'] = range(len(price_mean_by_region))

  # ラベルを元のデータにマージ
  train = train.merge(price_mean_by_region[['manufacturer', 'manufacturer_label']], on='manufacturer', how='left')
  test = test.merge(price_mean_by_region[['manufacturer', 'manufacturer_label']], on='manufacturer', how='left')

  # condition
  # 状態ごとに割り当てるラベルを定義
  condition_labels = {
      'excellent': 4,
      'good': 3,
      'like new': 5,
      'fair': 2,
      'new': 6,
      'salvage': 1,
  }

  # ラベルエンコーディングを実行
  train['condition_label'] = train['condition'].map(condition_labels)
  test['condition_label'] = test['condition'].map(condition_labels)

  # cylinders
  # 状態ごとに割り当てるラベルを定義
  condition_labels = {
      "6 cylinders": 6,
      "4 cylinders": 4,
      "8 cylinders": 8,
      "other": -30,
      "10 cylinders": 10,
      "5 cylinders": 5,
      "3 cylinders": 3,
      "12 cylinders": 12,
  }

  # ラベルエンコーディングを実行
  train['cylinders_label'] = train['cylinders'].map(condition_labels)
  test['cylinders_label'] = test['cylinders'].map(condition_labels)

  # fuel
  # 状態ごとに割り当てるラベルを定義
  condition_labels = {
      "gas": 2,
      "diesel": 5,
      "hybrid": 1,
      "other": 3,
      "electric": 4,
  }

  # ラベルエンコーディングを実行
  train['fuel_label'] = train['fuel'].map(condition_labels)
  test['fuel_label'] = test['fuel'].map(condition_labels)

  # titel_status
  # 状態ごとに割り当てるラベルを定義
  condition_labels = {
      "clean": 5,
      "rebuilt": 4,
      "salvage": 3,
      "lien": 6,
      "parts only": 2,
      "missing": 1,
  }

  # ラベルエンコーディングを実行
  train['title_status_label'] = train['title_status'].map(condition_labels)
  test['title_status_label'] = test['title_status'].map(condition_labels)

  # transmission
  # 状態ごとに割り当てるラベルを定義
  condition_labels = {
      "automatic": 3,
      "manual": 1,
      "other": 2,
  }

  # ラベルエンコーディングを実行
  train['transmission_label'] = train['transmission'].map(condition_labels)
  test['transmission_label'] = test['transmission'].map(condition_labels)

  # drive
  # 状態ごとに割り当てるラベルを定義
  condition_labels = {
      "fwd": 1,
      "4wd": 3,
      "rwd": 2,
  }

  # ラベルエンコーディングを実行
  train['drive_label'] = train['drive'].map(condition_labels)
  test['drive_label'] = test['drive'].map(condition_labels)

  # size
  # 状態ごとに割り当てるラベルを定義
  condition_labels = {
      'full-size': 4,
      'mid-size': 3,
      'compact': 1,
      'sub-compact': 2,
      'midーsize': 3,
      'fullーsize': 4,
      'mid−size': 3,
      'full−size': 4,
      'subーcompact': 2,
  }

  # ラベルエンコーディングを実行
  train['size_label'] = train['size'].map(condition_labels)
  test['size_label'] = test['size'].map(condition_labels)

  # type
  # 状態ごとに割り当てるラベルを定義
  condition_labels = {
      'truck': 13,
      'pickup': 12,
      'bus': 11,
      'van': 10,
      'offroad': 9,
      'SUV': 8,
      'convertible': 7,
      'other': 6,
      'coupe': 5,
      'mini-van': 4,
      'sedan': 3,
      'hatchback': 2,
      'wagon': 1,
  }

  # ラベルエンコーディングを実行
  train['type_label'] = train['type'].map(condition_labels)
  test['type_label'] = test['type'].map(condition_labels)

  # paint_color
  # 状態ごとに割り当てるラベルを定義
  condition_labels = {
      'white': 12,
      'black': 11,
      'yellow': 10,
      'custom': 9,
      'blue': 8,
      'grey': 7,
      'red': 6,
      'silver': 5,
      'orange': 4,
      'purple': 3,
      'brown': 2,
      'green': 1,
  }

  # ラベルエンコーディングを実行
  train['paint_color_label'] = train['paint_color'].map(condition_labels)
  test['paint_color_label'] = test['paint_color'].map(condition_labels)

  return train, test


In [ ]:
train, test = pre_process(train, test)

In [ ]:
#説明変数と目的変数に分ける
train_y = train.price
train_x = train.drop(columns=["price", "id", "region", "manufacturer", "condition", "cylinders", 'fuel', 'title_status', 'transmission', 'drive', 'size', 'type', 'paint_color', 'state'])
test_x = test.drop(columns=["id", "region", "manufacturer", "condition", "cylinders", 'fuel', 'title_status', 'transmission', 'drive', 'size', 'type', 'paint_color', 'state'])

#積と商によって特徴量作成
original_features = train_x.columns

for column in original_features:
  if column not in ["odometer"]:
    train_x['{}_div'.format(column)] = train_x["odometer"] / (train_x[column] + np.finfo(float).eps)
    test_x['{}_div'.format(column)] = test_x["odometer"] / (test_x[column] + np.finfo(float).eps)

for column in original_features:
  if column not in ["year", "odometer"]:
    train_x['{}_mul'.format(column)] = train_x["year"] * (train_x[column] + np.finfo(float).eps)
    test_x['{}_mul'.format(column)] = test_x["year"] * (test_x[column] + np.finfo(float).eps)

#欠損値を中央値で埋める
for col in train_x.columns:
  median = train_x[col].median()
  train_x[col].fillna(median, inplace=True)
  test_x[col].fillna(median, inplace=True)

#標準化
scaler = StandardScaler()
scaler.fit(train_x)

train_x = scaler.transform(train_x)
test_x = scaler.transform(test_x)

#DataFrame化
train_x = pd.DataFrame(train_x)
test_x = pd.DataFrame(test_x)

# 一層目

In [ ]:
class Model1NN:

    def __init__(self):
        self.model = None
        self.scaler = None

    def fit(self, tr_x, tr_y, va_x, va_y):

        batch_size = 64
        epochs = 200

        model = Sequential()

        model.add(Dense(256, input_shape=(train_x.shape[1],)))
        model.add(PReLU())
        model.add(BatchNormalization())
        model.add(Dropout(.1))

        model.add(Dense(256))
        model.add(PReLU())
        model.add(BatchNormalization())
        model.add(Dropout(.1))

        model.add(Dense(128))
        model.add(PReLU())
        model.add(BatchNormalization())
        model.add(Dropout(.05))

        model.add(Dense(64))
        model.add(PReLU())
        model.add(BatchNormalization())
        model.add(Dropout(.05))

        model.add(Dense(32))
        model.add(PReLU())
        model.add(BatchNormalization())
        model.add(Dropout(.05))

        model.add(Dense(16))
        model.add(PReLU())
        model.add(BatchNormalization())
        model.add(Dropout(.05))

        model.add(Dense(1))

        early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
        adam = optimizers.Adam(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
        model.compile(optimizer=adam, loss='mean_absolute_percentage_error', metrics=['accuracy'])



        history = model.fit(tr_x, tr_y, epochs=epochs, batch_size=batch_size, verbose=1, validation_data=(va_x, va_y), callbacks=[early_stopping])

        self.model = model

    def predict(self, x):
        pred = self.model.predict(x)
        return pred

class Model2NN:

    def __init__(self):
        self.model = None
        self.scaler = None

    def fit(self, tr_x, tr_y, va_x, va_y):

        batch_size = 64
        epochs = 100

        model = Sequential()

        model.add(Dense(512, input_shape=(train_x.shape[1],)))
        model.add(PReLU())
        model.add(BatchNormalization())
        model.add(Dropout(.1))

        model.add(Dense(256))
        model.add(PReLU())
        model.add(BatchNormalization())
        model.add(Dropout(.1))

        model.add(Dense(256))
        model.add(PReLU())
        model.add(BatchNormalization())
        model.add(Dropout(.1))

        model.add(Dense(128))
        model.add(PReLU())
        model.add(BatchNormalization())
        model.add(Dropout(.05))

        model.add(Dense(64))
        model.add(PReLU())
        model.add(BatchNormalization())
        model.add(Dropout(.05))

        model.add(Dense(32))
        model.add(PReLU())
        model.add(BatchNormalization())
        model.add(Dropout(.05))

        model.add(Dense(16))
        model.add(PReLU())
        model.add(BatchNormalization())
        model.add(Dropout(.05))

        model.add(Dense(1))

        early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
        adam = optimizers.Adam(lr=0.005, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
        model.compile(optimizer=adam, loss='mean_absolute_percentage_error', metrics=['accuracy'])



        history = model.fit(tr_x, tr_y, epochs=epochs, batch_size=batch_size, verbose=1, validation_data=(va_x, va_y), callbacks=[early_stopping])

        self.model = model

    def predict(self, x):
        pred = self.model.predict(x)
        return pred

class Model3NN:

    def __init__(self):
        self.model = None
        self.scaler = None

    def fit(self, tr_x, tr_y, va_x, va_y):

        batch_size = 64
        epochs = 100

        model = Sequential()

        model.add(Dense(256, input_shape=(train_x.shape[1],)))
        model.add(PReLU())
        model.add(BatchNormalization())
        model.add(Dropout(.1))

        model.add(Dense(16))
        model.add(PReLU())
        model.add(BatchNormalization())
        model.add(Dropout(.05))

        model.add(Dense(1))


        early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
        adam = optimizers.Adam(lr=0.005, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
        model.compile(optimizer=adam, loss='mean_absolute_percentage_error', metrics=['accuracy'])



        history = model.fit(tr_x, tr_y, epochs=epochs, batch_size=batch_size, verbose=1, validation_data=(va_x, va_y), callbacks=[early_stopping])

        self.model = model

    def predict(self, x):
        pred = self.model.predict(x)
        return pred

class Model4NN:

    def __init__(self):
        self.model = None
        self.scaler = None

    def fit(self, tr_x, tr_y, va_x, va_y):

        batch_size = 64
        epochs = 200

        model = Sequential()

        model.add(Dense(256, input_shape=(train_x.shape[1],)))
        model.add(PReLU())
        model.add(BatchNormalization())
        model.add(Dropout(.05))

        model.add(Dense(1))

        early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
        adam = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
        model.compile(optimizer=adam, loss='mean_absolute_percentage_error', metrics=['accuracy'])



        history = model.fit(tr_x, tr_y, epochs=epochs, batch_size=batch_size, verbose=1, validation_data=(va_x, va_y), callbacks=[early_stopping])

        self.model = model

    def predict(self, x):
        pred = self.model.predict(x)
        return pred

from sklearn.ensemble import RandomForestRegressor

class ModelRF:

    def __init__(self):
        self.model = None
        self.scaler = None

    def fit(self, tr_x, tr_y, va_x, va_y):

        batch_size = 64
        epochs = 100

        model = RandomForestRegressor(n_estimators=200, max_depth=10, random_state=42, verbose=0)
        model.fit(tr_x, tr_y)



        self.model = model

    def predict(self, x):
        pred = self.model.predict(x)
        return pred


def predict_cv(model, train_x, train_y, test_x):
    preds = []
    preds_test = []
    va_idxes = []

    kf = KFold(n_splits=5, shuffle=True, random_state=71)

    # クロスバリデーションで学習・予測を行い、予測値とインデックスを保存する
    for i, (tr_idx, va_idx) in enumerate(kf.split(train_x)):
        tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
        tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]
        model.fit(tr_x, tr_y, va_x, va_y)
        pred = model.predict(va_x)
        pred_in = model.predict(tr_x)
        print(mean_absolute_percentage_error(tr_y, pred_in))
        preds.append(pred)
        pred_test = model.predict(test_x)
        preds_test.append(pred_test)
        va_idxes.append(va_idx)

    # バリデーションデータに対する予測値を連結し、その後元の順序に並べ直す
    va_idxes = np.concatenate(va_idxes)
    preds = np.concatenate(preds, axis=0)
    order = np.argsort(va_idxes)
    pred_train = preds[order]

    # テストデータに対する予測値の平均をとる
    preds_test = np.mean(preds_test, axis=0)

    return pred_train, preds_test


model_1a = Model1NN()
pred_train_1a, pred_test_1a = predict_cv(model_1a, train_x, train_y, test_x)
pred_train_1a = pred_train_1a.flatten()
pred_test_1a = pred_test_1a.flatten()
print(mean_absolute_percentage_error(train_y, pred_train_1a))

model_1b = Model2NN()
pred_train_1b, pred_test_1b = predict_cv(model_1b, train_x_ps, train_y_ps, test_x)
pred_train_1b = pred_train_1b.flatten()
pred_test_1b = pred_test_1b.flatten()
print(mean_absolute_percentage_error(train_y, pred_train_1b))

model_1c = Model3NN()
pred_train_1c, pred_test_1c = predict_cv(model_1c, train_x_ps, train_y_ps, test_x)
pred_train_1c = pred_train_1c.flatten()
pred_test_1c = pred_test_1c.flatten()
print(mean_absolute_percentage_error(train_y, pred_train_1c))

model_1d = Model4NN()
pred_train_1d, pred_test_1d = predict_cv(model_1d, train_x, train_y, test_x)
pred_train_1d = pred_train_1d.flatten()
pred_test_1d = pred_test_1d.flatten()
print(mean_absolute_percentage_error(train_y, pred_train_1d))

In [ ]:
#既存の予測データの読み込み

# def move1index(df):
#   first_data = df.columns.values
#   # first_data[0] = float(first_data[0])
#   # first_data[1] = float(first_data[1])
#   df = df.values
#   df = np.vstack((first_data, df))
#   df = pd.DataFrame(df)
#   df = df.rename(columns={df.columns[0]: '', df.columns[1]: 'price'})
#   return np.array([float(x[1:-1]) for x in df['price'].values], dtype=np.float32)
#   # return df

# pred_train_1a = pd.read_csv("/content/drive/MyDrive/nn1_train.csv")
# pred_train_1b = pd.read_csv("/content/drive/MyDrive/nn2_train.csv")
# pred_train_1c = pd.read_csv("/content/drive/MyDrive/nn3_train.csv")
# pred_train_1d = pd.read_csv("/content/drive/MyDrive/nn4_train.csv")
# pred_test_1a = pd.read_csv("/content/drive/MyDrive/nn1_test.csv")
# pred_test_1b = pd.read_csv("/content/drive/MyDrive/nn2_test.csv")
# pred_test_1c = pd.read_csv("/content/drive/MyDrive/nn3_test.csv")
# pred_test_1d = pd.read_csv("/content/drive/MyDrive/nn4_test.csv")

# pred_train_1a = move1index(pred_train_1a)
# pred_train_1b = move1index(pred_train_1b)
# pred_train_1c = move1index(pred_train_1c)
# pred_train_1d = move1index(pred_train_1d)
# pred_test_1a = move1index(pred_test_1a)
# pred_test_1b = move1index(pred_test_1b)
# pred_test_1c = move1index(pred_test_1c)
# pred_test_1d = move1index(pred_test_1d)

hex_train_pred = pd.read_csv("/content/drive/MyDrive/train_an_lightGBM2.csv")
hex_test_pred = pd.read_csv("/content/drive/MyDrive/en_lightGBM2.csv")
# # hex_train_pred2 = pd.read_csv("/content/train_an_lightGBM_dart.csv")
# # hex_test_pred2 = pd.read_csv("/content/en_lightGBM_dart.csv")
# hex_train_pred = hex_train_pred["price"].values
hex_train_pred2 = hex_train_pred2["price"].values
hex_test_pred = hex_test_pred["price"].values
# # hex_test_pred2 = hex_test_pred2["price"].values

# 二層目

In [ ]:
# 2層目のモデル
# pred_train_2は、2層目のモデルの学習データのクロスバリデーションでの予測値
# pred_test_2は、2層目のモデルのテストデータの予測値

train_x_2_a = pd.DataFrame({'pred_1a': pred_train_1a, 'pred_1b': pred_train_1b, 'pred_1c': pred_train_1c, 'pred_1d': pred_train_1d, 'pred_1e': hex_train_pred})
test_x_2_a = pd.DataFrame({'pred_1a': pred_test_1a, 'pred_1b': pred_test_1b, 'pred_1c': pred_test_1c, 'pred_1d': pred_test_1d, 'pred_1e': hex_test_pred})

scaler = StandardScaler()
scaler.fit(train_x_2_a)

train_x_2_a = pd.DataFrame(scaler.transform(train_x_2_a))
test_x_2_a = pd.DataFrame(scaler.transform(test_x_2_a))


class Model2simple:

    def __init__(self):
        self.model = None
        self.scaler = None

    def fit(self, tr_x, tr_y, va_x, va_y):

        batch_size = 64
        epochs = 100

        model = Sequential()

        model.add(Dense(1, input_shape=(train_x_2_a.shape[1],)))
        # model.add(PReLU())
        # model.add(BatchNormalization())
        # model.add(Dropout(.1))

        # model.add(Dense(256))
        # model.add(PReLU())
        # model.add(BatchNormalization())
        # model.add(Dropout(.1))

        # model.add(Dense(128))
        # model.add(PReLU())
        # model.add(BatchNormalization())
        # model.add(Dropout(.05))

        # model.add(Dense(64))
        # model.add(PReLU())
        # model.add(BatchNormalization())
        # model.add(Dropout(.05))

        # model.add(Dense(32))
        # model.add(PReLU())
        # model.add(BatchNormalization())
        # model.add(Dropout(.05))

        # model.add(Dense(16))
        # model.add(PReLU())
        # model.add(BatchNormalization())
        # model.add(Dropout(.05))

        # model.add(Dense(1))


        early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
        adam = optimizers.Adam(lr=1, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
        model.compile(optimizer=adam, loss='mean_absolute_percentage_error', metrics=['accuracy'])



        history = model.fit(tr_x, tr_y, epochs=epochs, batch_size=batch_size, verbose=1, validation_data=(va_x, va_y), callbacks=[early_stopping])

        self.model = model

    def predict(self, x):
        pred = self.model.predict(x)
        return pred


class Model2LGBM:

    def __init__(self):
        self.model = None

    def fit(self, tr_x, tr_y, va_x, va_y):
        params = {
            'objective': 'mape',
            'metric': 'mape',
            'random_state': 42,
            'max_depth': 7,
            'n_estimators': 10000,
            'verbosity': -1,
            'early_stopping_round': 10,
        }
        num_round = 10
        dtrain = lgbm.Dataset(tr_x, label=tr_y)
        dvalid = lgbm.Dataset(va_x, label=va_y, reference=dtrain)
        self.model = lgbm.train(params, dtrain, num_round, valid_sets=[dtrain, dvalid])

    def predict(self, x):
        pred = self.model.predict(x)
        return pred


model_2 = Model2simple()
pred_train_2a, pred_test_2a = predict_cv(model_2, train_x_2_a, train_y, test_x_2_a)
pred_train_2a = pred_train_2a.flatten()
pred_test_2a = pred_test_2a.flatten()
print(mean_absolute_percentage_error(train_y, pred_train_2a))
# model_2 = Model1LGBM()
# pred_train_2b, pred_test_2b = predict_cv(model_2, train_x_2_a, train_y, test_x_2_a)
# print(mean_absolute_percentage_error(train_y, pred_train_2b))

In [ ]:
#レイヤーの重み
model_2.model.get_weights()

In [ ]:
#特徴量相関
train_x_2_a.corr()

In [ ]:
# utput_file = "二値分類_test.csv"
# with open(utput_file, 'w') as f:
#     for i in range(len(test)):
#         row = f"{test['id'][i]},{pred_test_2a[i]}\n"
#         f.write(row)

# utput_file = "二値分類_train.csv"
# with open(utput_file, 'w') as f:
#     for i in range(len(train)):
#         row = f"{train['id'][i]},{pred_train_2a[i]}\n"
#         f.write(row)

# 三層目

In [ ]:
# pred_train_2a = pd.read_csv("/content/drive/MyDrive/NNandLGBM_NN_train (1).csv")
# pred_test_2a = pd.read_csv("/content/drive/MyDrive/NNandLGBM_NN_test (1).csv")
hex_train_pred = pd.read_csv("/content/drive/MyDrive/train_en_light_nn_2.csv")
hex_test_pred = pd.read_csv("/content/drive/MyDrive/en_light_nn2.csv")
# hex_train_pred2 = pd.read_csv("/content/drive/MyDrive/train_an_lightGBM2.csv")
# hex_test_pred2 = pd.read_csv("/content/drive/MyDrive/en_lightGBM2.csv")

# first_data = katoro_train_pred.columns.values
# first_data[1] = float(first_data[1])
# katoro_train_pred = katoro_train_pred.values
# katoro_train_pred = np.vstack((first_data, katoro_train_pred))
# katoro_train_pred = pd.DataFrame(katoro_train_pred)
# katoro_train_pred = katoro_train_pred.rename(columns={0: '', 1: 'price'})

# first_data = katoro_test_pred.columns.values
# first_data[1] = float(first_data[1])
# katoro_test_pred = katoro_test_pred.values
# katoro_test_pred = np.vstack((first_data, katoro_test_pred))
# katoro_test_pred = pd.DataFrame(katoro_test_pred)
# katoro_test_pred = katoro_test_pred.rename(columns={0: '', 1: 'price'})

# pred_train_2a = pred_train_2a["price"].values
hex_train = hex_train_pred["price"].values
# pred_test_2a = pred_test_2a["price"].values
hex_test = hex_test_pred["price"].values
# hex_train2 = hex_train_pred2["price"].values
# hex_test2 = hex_test_pred2["price"].values

In [ ]:
train_x_3 = pd.DataFrame({'pred_1d': pred_train_2a, 'pred_1e': hex_train})
test_x_3 = pd.DataFrame({'pred_1d': pred_test_2a, 'pred_1e': hex_test})

scaler = StandardScaler()
scaler.fit(train_x_3)

train_x_3 = pd.DataFrame(scaler.transform(train_x_3))
test_x_3 = pd.DataFrame(scaler.transform(test_x_3))


class Model3simple:

    def __init__(self):
        self.model = None
        self.scaler = None

    def fit(self, tr_x, tr_y, va_x, va_y):

        batch_size = 64
        epochs = 300

        model = Sequential()

        # model.add(Dense(32, input_shape=(train_x_2.shape[1],)))
        # model.add(PReLU())
        model.add(Dense(1))

        early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
        adam = optimizers.Adam(lr=0.2, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
        sgd = optimizers.SGD(learning_rate=0.03)
        model.compile(optimizer=adam, loss='mean_absolute_percentage_error', metrics=['accuracy'])



        history = model.fit(tr_x, tr_y, epochs=epochs, batch_size=batch_size, verbose=1, validation_data=(va_x, va_y), callbacks=[early_stopping])

        self.model = model

    def predict(self, x):
        pred = self.model.predict(x)
        return pred

model_3 = Model3simple()
pred_train_3, pred_test_3 = predict_cv(model_3, train_x_3, train_y, test_x_3)
pred_train_3 = pred_train_3.flatten()
pred_test_3 = pred_test_3.flatten()
print(mean_absolute_percentage_error(train_y, pred_train_3))

In [ ]:
#レイヤー重み
model_3.model.get_weights()

In [ ]:
#特徴量相関
train_x_3.corr()

In [ ]:
# 最終予測出力
# output_file = "うんち博打二号.csv"
# with open(output_file, 'w') as f:
#     for i in range(len(test)):
#         row = f"{test['id'][i]},{pred_test_2[i]}\n"
#         f.write(row)